<div align="right">  
    Grupo 13  
    
    André Neves da Costa - A95869 
    Filipe José Silva Castro - A96156 
</div>

## Variáveis de input:
+ *q*, *m*, *n* $\rightarrow$  Números inteiros

## Variáveis auxiliares:
+ *d* $\rightarrow$ Números inteiro dependente de q
+ *L* $\rightarrow$ Matriz aleatória com dimensões *m*(linhas) $\times$ *n*(colunas) representada através de um dicionário cujas keys são tuplos (j,i) correspondendo j às linhas e i às colunas
+ *e* $\rightarrow$ lista onde se vão armazenar os valores do vetor que satisfaz o problema

## Condições:
1. *e* não pode ser um vetor nulo
2. $ q \ge 3 $
3. $m > n$
4. $|m| > 1 + |n|$
5. $|q| > |m|$
6. *q* é um número primo
7. $\forall\,i < n\,\centerdot\,\sum_{j< m}\,e_j\,\times\,\mathsf{L}_{j,i}\;\equiv\;0\mod q$

## Critérios de otimização
1. Minimizar o numero de componentes não nulos de *e*

## Implementação em python:

In [232]:
import random
from ortools.sat.python import cp_model

#q,m,n = 67,32,8
#q,m,n = 37,16,7
#q,m,n = 37,16,6


#q,m,n = 37,16,5

#    Vetor e = [1, 0, -1, 1, 1, 0, 0, 0, 1, -1, 0, 0, -1, 0, 1, -1]

#    Matriz L= 
#    [12, 2, 0, 11, 13]
#    [-11, -1, 4, -8, 8]
#    [16, -6, -11, 9, -3]
#    [-11, -1, 0, 10, -6]
#    [-1, 2, -17, 10, 14]
#    [-13, 17, -15, -5, -2]
#    [-13, 17, 11, -2, -14]
#    [-5, 0, 18, 4, -7]
#    [13, 0, -5, -6, 11]
#    [18, -13, 15, 3, -1]
#    [12, -12, 3, -5, -3]
#    [-18, -7, 1, 1, 5]
#    [-6, 2, -9, 17, 9]
#    [5, 16, -14, 9, -12]
#    [9, -16, 0, -5, -12]
#    [-6, 4, -17, -9, 15]

#q,m,n = 37,16,4

#    Vetor e = [0, 1, -1, 1, -1, -1, 1, 1, 0, 1, 0, -1, 1, 1, 0, 1]

#    Matriz L= 
#    [6, 15, -5, 14]
#    [-13, 0, 10, 16]
#    [16, -8, 14, 12]
#    [14, 2, 0, -12]
#    [6, -10, -18, -15]
#    [-16, 15, 12, 3]
#    [-17, 2, 8, -4]
#    [-10, -10, 9, 15]
#    [-14, -4, 5, -17]
#    [-8, 8, -15, 18]
#    [-14, 7, -15, -3]
#    [-11, 3, 10, 18]
#    [12, -17, 5, -12]
#    [5, 15, -5, -1]
#    [-17, -17, -14, 11]
#    [12, 0, 6, -2]

q,m,n = 17,8,3

#    Vetor e = [-1, 0, -1, 1, 0, -1, 1, 0]

#    Matriz L= 
#    [0, -8, -5]
#    [-8, 2, 6]
#    [-8, -1, -5]
#    [-6, 4, 5]
#    [1, 3, 1]
#    [-8, -5, 1]
#    [7, -1, 3]
#    [-8, 1, 2]

#q,m,n = 17,8,2

#    Vetor e = [0, 1, 0, -1, 0, 0, 1, 0]

#    Matriz L= 
#    [-3, 7]
#    [0, 6]
#    [3, 7]
#    [1, 3]
#    [-6, 8]
#    [-1, 5]
#    [1, -3]
#    [1, 4]

#q,m,n = 8,4,1

#    Vetor e = [1, 0, 0, 0]

#    Matriz L= 
#    [0]
#    [-2]
#    [-1]
#    [-3]

d = int((q-1)/2)

L = {}

for j in range(m):
    for i in range(n):
       L[(j,i)] = random.randint(-d,d)
        

model = cp_model.CpModel()      

e = []

for i in range(m):
    e.append(model.NewIntVar(-1,1,f'[{i}]'))

1. e não pode ser um vetor nulo
$$\sum_{j<m} (e_{j})^2 \ge 1$$

In [ ]:
ee = []

for i in range(m):
    ee.append(model.NewIntVar(0, 1, f'e[{i}]*e[{i}]'))
    model.AddMultiplicationEquality(ee[i], [e[i],e[i]])

model.Add(sum([ee[j] for j in range(m)]) >= 1)

2. $$ q \ge 3 $$

In [ ]:
model.Add(q >= 3)

3. $$m > n$$

In [ ]:
model.Add(m > n)

4. $$|m| > 1 + |n|$$

In [ ]:
model.Add(len(bin(m)) > 1+len(bin(n)))

5. $$|q| > |m|$$

In [ ]:
model.Add(len(bin(q)) > len(bin(m)))

6. $$\forall_{1<i<q}\,i\;\not\equiv\;0\mod q$$

In [238]:
for i in range(2,q):
    model.Add(i%q != 0)

7. $$\forall\,i < n\,\centerdot\,\sum_{j< m}\,e_j\,\times\,\mathsf{L}_{j,i}\;\equiv\;0\mod q$$

In [239]:
M={}

for i in range(n):
    for j in range(m):
        M[j,i]=model.NewIntVar(-d,d,f'M[{j,i}]')
        
for j in range(m):
    for i in range(n):
        model.AddMultiplicationEquality(M[j,i],[e[j],L[j,i]])
    
msom={}
for j in range(m):
    msom[j]=model.NewIntVar(-m*d,m*d,f'msom[{j}]')

for i in range(n):
    model.Add(sum([M[j,i] for j in range(m)]) == msom[j])
    
x={}
for j in range(m):
    x[j]=model.NewIntVar(0,q,f'M[{j,i}]')
    model.AddModuloEquality(x[j],msom[j],q)
    
for j in range(m):
    model.Add(x[j] == 0)

8. Minimizar o número de componentes não nulos de e

In [240]:
model.Minimize(sum([ee[j] for j in range(m)]))

## Imprimir o resultado:

In [ ]:
solver = cp_model.CpSolver()

status = solver.Solve(model)
resultado=[]    
    
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for j in range(m):
        resultado.append(solver.Value(e[j]))
    print("\nVetor e =",resultado)
else:
    print('\nNão existe solução')

print("\nMatriz L= ")
for j in range(m):
    resultado=[]
    for i in range(n):
        resultado.append(L[(j,i)])
    print(resultado)